In [38]:
import pickle
import pymongo
import pandas as pd
from bson import ObjectId

In [39]:
with open('et_classifier.pkl', 'rb') as f:
    et_classifier = pickle.load(f)
with open('gb_classifier.pkl', 'rb') as f:
    gb_classifier = pickle.load(f)
with open('hgb_classifier.pkl', 'rb') as f:
    hgb_classifier = pickle.load(f)
with open('rf_classifier.pkl', 'rb') as f:
    rf_classifier = pickle.load(f)
with open('ab_classifier.pkl', 'rb') as f:
    ab_classifier = pickle.load(f)
with open('ann.pkl', 'rb') as f:
    ann = pickle.load(f)
with open('standart_scaler.pkl', 'rb') as f:
    sc = pickle.load(f)

In [40]:
MONGO_CLIENT = pymongo.MongoClient("mongodb://192.168.1.7:27017/")
SDA_DB = MONGO_CLIENT["steam_dota_api"]
LGC = SDA_DB["league_games_col"]

In [41]:
# matches = MLRS_COL.find({"_id": ObjectId("65b5e65abb22a1e961d58c20")})
matches = LGC.find({"match_id": 7562523900})

matches_for_pd = []

for match in matches:
    match_dict = {}

    match_dict['game_time'] = int(match['scoreboard']['duration'])
    match_dict['score'] = match["scoreboard"]['radiant']["score"] - match["scoreboard"]['dire']["score"]

    networth = 0
    for player in match['scoreboard']['radiant']['players']:
        networth += player['net_worth']
    for player in match['scoreboard']['dire']['players']:
        networth -= player['net_worth']
    match_dict['net_worth'] = networth

    for p_idx, player in enumerate(match['scoreboard']['radiant']['players']):
        match_dict["team_0_player_{}_level".format(p_idx)] = player['level']
        match_dict["team_0_player_{}_kill_count".format(p_idx)] = player['kills']
        match_dict["team_0_player_{}_death_count".format(p_idx)] = player['death']
        match_dict["team_0_player_{}_assists_count".format(p_idx)] = player['assists']
        match_dict["team_0_player_{}_denies_count".format(p_idx)] = player['denies']
        match_dict["team_0_player_{}_lh_count".format(p_idx)] = player['last_hits']
        match_dict["team_0_player_{}_gold".format(p_idx)] = player['gold']
        match_dict["team_0_player_{}_net_worth".format(p_idx)] = player['net_worth']

        match_dict["team_0_player_{}_item_0".format(p_idx)] = player['item0']
        match_dict["team_0_player_{}_item_1".format(p_idx)] = player['item1']
        match_dict["team_0_player_{}_item_2".format(p_idx)] = player['item2']
        match_dict["team_0_player_{}_item_3".format(p_idx)] = player['item3']
        match_dict["team_0_player_{}_item_4".format(p_idx)] = player['item4']
        match_dict["team_0_player_{}_item_5".format(p_idx)] = player['item5']

    for p_idx, player in enumerate(match['scoreboard']['dire']['players']):
        match_dict["team_1_player_{}_level".format(p_idx)] = player['level']
        match_dict["team_1_player_{}_kill_count".format(p_idx)] = player['kills']
        match_dict["team_1_player_{}_death_count".format(p_idx)] = player['death']
        match_dict["team_1_player_{}_assists_count".format(p_idx)] = player['assists']
        match_dict["team_1_player_{}_denies_count".format(p_idx)] = player['denies']
        match_dict["team_1_player_{}_lh_count".format(p_idx)] = player['last_hits']
        match_dict["team_1_player_{}_gold".format(p_idx)] = player['gold']
        match_dict["team_1_player_{}_net_worth".format(p_idx)] = player['net_worth']

        match_dict["team_1_player_{}_item_0".format(p_idx)] = player['item0']
        match_dict["team_1_player_{}_item_1".format(p_idx)] = player['item1']
        match_dict["team_1_player_{}_item_2".format(p_idx)] = player['item2']
        match_dict["team_1_player_{}_item_3".format(p_idx)] = player['item3']
        match_dict["team_1_player_{}_item_4".format(p_idx)] = player['item4']
        match_dict["team_1_player_{}_item_5".format(p_idx)] = player['item5']

    matches_for_pd.append(match_dict)

In [42]:
df = pd.DataFrame(matches_for_pd)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Columns: 143 entries, game_time to team_1_player_4_item_5
dtypes: int64(143)
memory usage: 41.5 KB


In [43]:
print(df.columns)

Index(['game_time', 'score', 'net_worth', 'team_0_player_0_level',
       'team_0_player_0_kill_count', 'team_0_player_0_death_count',
       'team_0_player_0_assists_count', 'team_0_player_0_denies_count',
       'team_0_player_0_lh_count', 'team_0_player_0_gold',
       ...
       'team_1_player_4_denies_count', 'team_1_player_4_lh_count',
       'team_1_player_4_gold', 'team_1_player_4_net_worth',
       'team_1_player_4_item_0', 'team_1_player_4_item_1',
       'team_1_player_4_item_2', 'team_1_player_4_item_3',
       'team_1_player_4_item_4', 'team_1_player_4_item_5'],
      dtype='object', length=143)


In [44]:
X = df.iloc[:, :].values
print(X)

[[   7    0  190 ...   44   16   -1]
 [  67   -1 -139 ...   44   16   -1]
 [ 127   -1 -338 ...   44   16   -1]
 ...
 [1634    5 3196 ...    1  232   -1]
 [1694    5 3894 ...    1  232   -1]
 [1755    6 6954 ...    1  232  188]]


In [45]:
# print(X[-4])

In [46]:
for pred in et_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

radiant 0.57
radiant 0.51
dire 0.57
radiant 0.55
dire 0.61
dire 0.59
dire 0.58
dire 0.53
dire 0.61
dire 0.59
dire 0.66
dire 0.68
dire 0.7
dire 0.75
dire 0.75
dire 0.75
dire 0.75
dire 0.75
dire 0.75
dire 0.75
dire 0.77
dire 0.75
dire 0.73
dire 0.66
dire 0.67
dire 0.68
dire 0.67
dire 0.71
dire 0.73
dire 0.57
dire 0.52
dire 0.53
radiant 0.51
radiant 0.55
radiant 0.61
radiant 0.66
radiant 0.67


In [47]:
for pred in gb_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

dire 0.73
dire 0.65
dire 0.8
dire 0.58
dire 0.81
dire 0.72
dire 0.74
dire 0.68
dire 0.79
dire 0.55
radiant 0.6
dire 0.79
dire 0.66
dire 0.96
dire 0.96
dire 0.96
dire 0.96
dire 0.96
dire 0.96
dire 0.96
dire 0.85
dire 0.62
dire 0.51
radiant 0.51
radiant 0.51
dire 0.52
radiant 0.5
dire 0.85
dire 0.89
dire 0.82
dire 0.7
dire 0.81
radiant 0.8
radiant 0.57
dire 1.0
radiant 0.68
radiant 0.78


In [48]:
for pred in hgb_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

dire 0.67
radiant 0.5
radiant 0.57
dire 0.54
dire 0.61
dire 0.58
dire 0.6
dire 0.58
dire 0.79
dire 0.83
dire 0.66
dire 0.68
dire 0.73
dire 0.92
dire 0.92
dire 0.92
dire 0.92
dire 0.92
dire 0.92
dire 0.92
dire 0.93
dire 0.86
dire 0.83
dire 0.83
dire 0.75
dire 0.81
dire 0.82
dire 0.94
dire 0.96
dire 0.95
dire 0.92
dire 0.93
dire 0.83
dire 0.78
dire 0.7
dire 0.54
radiant 0.81


In [49]:
for pred in rf_classifier.predict_proba(X):
    if pred[0] > pred[1]:
        print('dire', round(pred[0], 2))
    else:
        print('radiant', round(pred[1], 2))

radiant 0.58
radiant 0.54
dire 0.64
radiant 0.5
dire 0.62
dire 0.72
dire 0.54
dire 0.54
dire 0.56
dire 0.68
dire 0.78
dire 0.68
dire 0.8
dire 0.82
dire 0.82
dire 0.82
dire 0.82
dire 0.82
dire 0.82
dire 0.82
dire 0.84
dire 0.78
dire 0.82
dire 0.62
dire 0.66
dire 0.72
dire 0.76
dire 0.74
dire 0.68
dire 0.7
dire 0.72
dire 0.64
radiant 0.52
dire 0.52
dire 0.52
radiant 0.56
radiant 0.6


In [50]:
X_ss = sc.transform(X)

y_pred = ann.predict(X_ss)
for y in y_pred:
    if y[0] > 0.5:
        print('radiant', round(y[0], 2))
    else:
        print('dire', round(1- y[0], 2))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


2/2 [==============================] - 0s 4ms/step
radiant 0.98
dire 0.72
dire 0.98
radiant 1.0
dire 0.85
dire 0.93
radiant 1.0
radiant 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
dire 1.0
radiant 1.0
